In [1]:
import numpy as np
from IPython.display import HTML
from pydrake.all import (
    AbstractValue,
    ConstantValueSource,
    ConstantVectorSource,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    BasicVector,
    LeafSystem,
    MeshcatVisualizer,
    MathematicalProgram,
    RandomGenerator,
    RigidTransform,
    UniformlyRandomRotationMatrix,
    Simulator,
    SnoptSolver,
    StartMeshcat,
    ge,
    le,
)

from pydrake.systems.framework import GenerateHtml

In [2]:
from manipulation.exercises.robot.test_hardware_station_io import (
    TestHardwareStationIO,
)
from manipulation.scenarios import AddIiwaDifferentialIK
from manipulation.station import MakeHardwareStation, load_scenario

In [3]:
meshcat = StartMeshcat()
rng = np.random.default_rng(145)  # this is for python
generator = RandomGenerator(rng.integers(0, 1000))  # this is for c++

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [4]:
import os

current_directory = os.getcwd()

scenario_data = f"""
directives:
- add_directives:
    file: file:///{current_directory}/connect4-assets/robots.yaml
- add_model:
    name: connect4
    file: file:///{current_directory}/connect4-assets/connect4-convex.sdf
- add_weld:
    parent: world
    child: connect4
    X_PC:
        translation: [0, -0.25, 0]
        
# - add_model:
#     name: table_top
#     file: package://manipulation/table_top.sdf
# - add_weld:
#     parent: world
#     child: table_top::table_top_center
"""


NUM_CHIPS = 7
for i in range(1, NUM_CHIPS+1):
  height = (i - 1) * 0.1
  offset = 0.0825 * (i - 4)
  
  scenario_data += f"""
- add_model:
    name: red_chip_{i}
    file: file:///{current_directory}/connect4-assets/red_chip.sdf
    default_free_body_pose:
        red_chip:
            translation: [{offset}, 0.022, 0.6]
            rotation: !Rpy
                deg: [90, 0, 0]
                
  """
    
scenario_data += """
model_drivers:
    iiwa1: !IiwaDriver
      hand_model_name: wsg1
    wsg1: !SchunkWsgDriver {}
    iiwa2: !IiwaDriver
      hand_model_name: wsg2
    wsg2: !SchunkWsgDriver {}
"""


In [5]:
class PoseSource(LeafSystem):
    def __init__(self, pose):
        LeafSystem.__init__(self)
        self._pose = pose
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcOutput
        )

    def CalcOutput(self, context, output):
        output.set_value(self._pose)
        
    def set_pose(self, new_pose):
        self._pose = new_pose

In [6]:
# Build diagram
builder = DiagramBuilder()
scenario = load_scenario(data=scenario_data)

station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))
plant = station.GetSubsystemByName("plant")

controller_plant_1 = station.GetSubsystemByName(
    "iiwa1.controller"
).get_multibody_plant_for_control()

controller_plant_2 = station.GetSubsystemByName(
    "iiwa2.controller"
).get_multibody_plant_for_control()

pose1 = RigidTransform()
pose1.set_translation([-0.25, -0.25, 0.5])
pose1_source = builder.AddSystem(PoseSource(pose1))
controller1 = AddIiwaDifferentialIK(
    builder,
    controller_plant_1, 
    frame=controller_plant_1.GetFrameByName("body")
)

builder.Connect(
    pose1_source.get_output_port(),
    controller1.get_input_port(0),
)

builder.Connect(
    station.GetOutputPort("iiwa1.state_estimated"),
    controller1.GetInputPort("robot_state"),
)

builder.Connect(
    controller1.get_output_port(),
    station.GetInputPort("iiwa1.position"),
)

pose2_source = builder.AddSystem(PoseSource(pose1))
controller2 = AddIiwaDifferentialIK(
    builder,
    controller_plant_2, 
    frame=controller_plant_2.GetFrameByName("body")
)

builder.Connect(
    pose2_source.get_output_port(),
    controller2.get_input_port(0),
)

builder.Connect(
    station.GetOutputPort("iiwa2.state_estimated"),
    controller2.GetInputPort("robot_state"),
)

builder.Connect(
    controller2.get_output_port(),
    station.GetInputPort("iiwa2.position"),
)

wsg1_position = builder.AddSystem(ConstantVectorSource([0.1]))
builder.Connect(
    wsg1_position.get_output_port(),
    station.GetInputPort("wsg1.position"),
)

wsg2_position = builder.AddSystem(ConstantVectorSource([0.1]))
builder.Connect(
    wsg2_position.get_output_port(),
    station.GetInputPort("wsg2.position"),
)

visualizer = MeshcatVisualizer.AddToBuilder(
    builder, station.GetOutputPort("query_object"), meshcat
)

diagram = builder.Build()

In [7]:
gripper_frame_1 = controller_plant_1.GetFrameByName("body")
gripper_frame_2 = controller_plant_2.GetFrameByName("body")
world_frame = plant.world_frame()

# print(world_frame)

# Set Context
context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(
    plant, context
)

# print(context)

# provide initial states
q0 = np.array(
    [
        1.40666193e-05,
        1.56461165e-01,
        -3.82761069e-05,
        -1.32296976e00,
        -6.29097287e-06,
        1.61181157e00,
        -2.66900985e-05,
    ]
)

# set the joint positions of both kuka arms
iiwa1 = plant.GetModelInstanceByName("iiwa1")
plant.SetPositions(plant_context, iiwa1, q0)
plant.SetVelocities(plant_context, iiwa1, np.zeros(7))
wsg1 = plant.GetModelInstanceByName("wsg1")
plant.SetPositions(plant_context, wsg1, [-0.05, 0.05])
plant.SetVelocities(plant_context, wsg1, [0, 0])

iiwa2 = plant.GetModelInstanceByName("iiwa2")
plant.SetPositions(plant_context, iiwa2, q0)
plant.SetVelocities(plant_context, iiwa2, np.zeros(7))
wsg2 = plant.GetModelInstanceByName("wsg2")
plant.SetPositions(plant_context, wsg2, [-0.05, 0.05])
plant.SetVelocities(plant_context, wsg2, [0, 0])    

diagram.ForcedPublish(context)

In [8]:
simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5);

In [9]:
systems = diagram.GetSystems()

diff_ik_source = systems[1]
pose = RigidTransform()
pose.set_translation([-0.55, -0.55, 0.5])

diff_ik_source.set_pose(pose)

In [10]:
simulator.AdvanceTo(20);